In [205]:
#CARGAR INDOMIO
import pandas as pd
import numpy as np
import re
from unidecode import unidecode
from functools import reduce
import ast
from unidecode import unidecode
from sklearn.preprocessing import MultiLabelBinarizer

In [206]:
pd.set_option("display.max_columns", 500)
pd.set_option('display.max_rows',None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_colwidth', None)
# Ajustar la configuración para mostrar más filas 
#pd.set_option('display.max_rows', 500)

#inicializo la lista to replace
to_replace=['Null','Nan','nan','NULL',None]


In [207]:
# Función para extraer texto en mayúsculas 
def extraer_mayusculas(texto): 
    if isinstance(texto, str):
     return ' '.join(re.findall(r'\b[A-Z]+\b', texto))
    return ''
 # Aplicar la función a la columna 'texto' 

In [208]:
# Función para normalizar texto (eliminar acentos y convertir a minúsculas)
def normalizar_texto(texto):
 return unidecode(texto).lower()

In [209]:
# Función para quitar determinantes
def quitar_determinantes(texto):
    determinantes = ['el', 'la', 'los', 'las', 'un', 'una', 'unos', 'unas']
    palabras = texto.split()
    palabras_sin_determinantes = [palabra for palabra in palabras if palabra.lower() not in determinantes]
    return ' '.join(palabras_sin_determinantes)

In [210]:
dfi= pd.read_csv('C:/Mas/Dataset_Proyecto/Ficheros_Ventas/Indomio/scraping_indomio.csv')
dfdistritos= pd.read_csv('C:/Mas/Dataset_Proyecto/Ficheros_Ventas/Indomio/poblacion_1_enero.csv',sep=";")


In [211]:
dfi["badges"].head

<bound method NDFrame.head of 0                                                                          ['Cancha de tenis', 'Exterior', 'Fibra óptica', 'Puerta blindada', 'Instalación tv centralizada', 'Piscina', 'Videoportero']
1                                                                            ['Carpintería exterior de doble vidrio/PVC', 'Fibra óptica', 'Interior', 'Puerta blindada', 'Instalación tv centralizada']
2                                                                                                                                                                                                    []
3                                                                                                                                                                                                    []
4                                                                                                                                                                         

In [212]:
#me quedo solo con las columnas 'cod_distrito', 'distrito','cod_barrio', 'barrio'
# Eliminar las filas donde la columna 'distrito' es igual a la columna 'barrio'
dfd=dfdistritos[['cod_distrito', 'distrito','cod_barrio', 'barrio']]
dfd=dfd.iloc[21:]
#eliminar la fila con distrito Todos
dfd=dfd.query("distrito!='Todos'")
#ante de hacer nada modifico algunos nombres que se han cargado mal

# Asegurarse de que no haya espacios adicionales y que todo esté en minúsculas

dfd.loc[:, 'distrito_n'] = [normalizar_texto(valor.strip().replace(" ", "")) for valor in dfd.loc[:, 'distrito']]
dfd.loc[:, 'barrio_n'] = [normalizar_texto(valor.strip().replace(" ", "")) for valor in dfd.loc[:, 'barrio']]

#eliminar duplicados
dfd=dfd.drop_duplicates()
#reseteo el indice
dfd.reset_index(drop=True)
#creo lista de distritos y barrios

lista_distritos=dfd["distrito"].unique()
lista_barrios=dfd["barrio"].unique()
'quito los espacios de todos los nombres y normalizo'

lista_distritos = [quitar_determinantes(normalizar_texto(valor.strip().replace(" ", ""))) for valor in lista_distritos]
lista_barrios = [quitar_determinantes(normalizar_texto(valor.strip().replace(" ", ""))) for valor in lista_barrios]

dfd.to_feather("df_distritos.ftr")

In [213]:
def eliminar_duplicados(row):
    if row['atr'] is not None:
        valores = row['atr'].split('|')
        valores_unicos = []
        valores_normalizados = set()
        for valor in valores:
            valor_normalizado = valor.replace(' ', '').lower()
            if valor_normalizado not in valores_normalizados:
                valores_unicos.append(valor)
                valores_normalizados.add(valor_normalizado)
        return '|'.join(valores_unicos)
    return row['atributos']

In [214]:
# La columna con los valores es 'atributos'
def procesar_atributos(fila):
    if isinstance(fila, str):  # Verificar si la fila es una cadena de texto
        fila = fila.strip('|')  # Eliminar separadores '|' al principio y al final
        atributos = fila.split('|')
        datos = {}
        for atributo in atributos:
            partes = atributo.split('###', 1)  # Dividir en dos partes como máximo
            if len(partes) == 2:
                nombre, valor = partes
                datos[nombre.strip()] = valor.strip()
            else:
                nombre = partes[0].strip()                
                if pd.isna(nombre): 
                    nombre = ''
                if nombre:
                    datos[nombre] = 1
        return datos
    return {}

In [215]:
# Crear la nueva columna 'sup_comercial'
def obtener_sup_comercial(row,df):
    for col in df.columns:
        if 'comercial' in col and row[col] == 1:
            try:
                return col.split()[1]
            except IndexError:
                return None
    return None

In [216]:
# Crear la nueva columna 'tipo_inmueble'
def obtener_tipo_inmueble(row,df):
    for col in df.columns:
        if 'Tipo de inmueble' in col and row[col] == 1:
            try:
                return col.split()[3]
            except IndexError:
                return None
    return None

In [217]:
def esbarrio_o_distrito(nombre):
    # Normalizar el nombre
      #lo trato apra que sea comparable con la lista
    nnombre = quitar_determinantes(normalizar_texto(nombre.strip().replace(" ", "")))
   
    # Inicializar variables
    lesdistrito = False
    lesbarrio = False
    zona=nombre
    dist = "Desconocido"
    cod_dist = 0
    bar = "Desconocido"
    cod_bar = 0
    
    # Comprobar si es un distrito
    for d in lista_distritos:
       # dist_n = normalizar_texto(d)       
        if nnombre == d:
            lesdistrito = True
            dist=nombre
            try:
                 # Traer el valor del número de distrito 
                cod_dist = dfd.query(f"distrito_n == '{nnombre}'")["cod_distrito"].values[0] 
                
            except IndexError: 
                #cod_dist = "No Informado" 
                cod_dist = "Error traer codigo" 
                #bar = "No Informado" 
                #cod_bar = 0 break
            # Traer el valor del número de distrito
            #cod_dist = dfd.query(f"distrito == '{dist_n}'")["cod_distrito"].values[0]
            bar = "desconocido"
            cod_bar = 0
            break
    
    # Si no es un distrito, comprobar si es un barrio
    if not lesdistrito:
        for b in lista_barrios:
           # bar_n = normalizar_texto(b)          
            if nnombre == b:
                lesbarrio = True
                bar=nombre
                try:
                 # Traer el valor del número de distrito y barrio 
                    cod_dist = dfd.query(f"barrio_n == '{nnombre}'")["cod_distrito"].values[0]
                    cod_bar = dfd.query(f"barrio_n == '{nnombre}'")["cod_barrio"].values[0]
                    dist = dfd.query(f"barrio_n == '{nnombre}'")["distrito"].values[0] 
                except IndexError: 
                    cod_dist = 0
                    cod_bar = 0
                    dist = "Error encuentra barrio"
                break
                # Traer el valor del número de distrito y barrio
               # cod_dist = dfd.query(f"barrio == '{bar_n}'")["cod_distrito"].values[0]
                #cod_bar = dfd.query(f"barrio == '{bar_n}'")["cod_barrio"].values[0]
                #dist = dfd.query(f"barrio == '{bar_n}'")["distrito"].values[0]
                #break
    
    return pd.Series([dist, cod_dist, bar, cod_bar,zona])


In [218]:
def procesar_direccion(fila):
    # Distritos cuyos nombres oficiales tienen un -, y barrios
    exc = ['Fuencarral-El Pardo', 'Moncloa-Aravaca', 'San Blas-Canillejas', 'Palomas-Piovera', 'Simancas-Hellín', 'Arcos-Rosas', 'Rejas-Canillejas', 'Salvador-Canillejas']
    sep = '-'
    title2 = ""
    zona = "desconocida"
    
    # Miro si la fila tiene -, si lo tiene y no está contenido en exc, lo separo, la primera parte 
    # lo paso a la función esbarrio_o_distrito y la segunda parte lo voy a poner en una columna zona
    
    try:
        partes = fila.split(',')
        if len(partes) >= 2:
            ciudad = partes[-1].strip()
            nombre = partes[-2].strip()
            #san andres no es un barrio, es una zona dentro de villaverde alto, igual que sanchinarro de hortaleza

          #  if nombre=="san andres": 
             #   nombre="villaverdealto-sanandres"
            if sep in nombre and nombre not in exc:
                # Separar la primera parte y la segunda parte
                pparte, zona = nombre.split(sep, 1)
                # Ver si nombre es distrito o barrio y devolver los valores correspondientes
                distrito, cod_dist, barrio, cod_barrio,zona = esbarrio_o_distrito(pparte)
            else:
                distrito, cod_dist, barrio, cod_barrio,zona = esbarrio_o_distrito(nombre)
            if len(partes) >= 3:
                title2 = ','.join(partes[:-2]).strip()
            return pd.Series([title2, ciudad, distrito, cod_dist, barrio, cod_barrio, zona])
        
    except Exception as e:
        # Manejar el error y devolver valores predeterminados
        print(f"Error al procesar la dirección: {e}")
        return pd.Series([fila, "Error procesar direccion", "No Informado", 0, "No Informado", 0, "desconocida"])
    
    return pd.Series([None, None, None, None, None, None, None])




In [219]:
#  DataFrame se llama dfi y la columna con los valores es 'title2'
def extraer_calle(fila):
    fila_normalizada = normalizar_texto(fila)
    patrones = ['calle', 'plaza', 'avenida', 'glorieta']
    for patron in patrones:
        if patron in fila_normalizada:
            return fila_normalizada[fila_normalizada.index(patron):]
    return None

In [220]:
def asignar_zona_distrito_barrio(row):
    if row['zona'] == 'Sanchinarro':
        row['distrito'] = 'Hortaleza'
        row['barrio'] = 'Valdefuentes'
    if row['zona'] == 'San Andrés':
        row['distrito'] = 'Villaverde'
        row['barrio'] = 'VillaverdeAlto,CascoHistóricodeVillaverde'
    if row['zona'] == 'Virgen del Cortijo':
        row['distrito'] = 'Hortaleza'
        row['barrio'] = 'Valdefuentes'
    if row['zona'] == 'Montecarmelo':
        row['distrito'] = 'Fuencarral-El Pardo'
        row['barrio'] = 'Mirasierra'

    return row

In [221]:
#Elimino las lineas en el que el precio es Precio a consultar	
#Eliminar las filas donde la columna 'precio' es nula
dfi = dfi.dropna(subset=['precio'])
dfi=(dfi.query("precio!='Precio a consultar'"))
dfi['precio'] = dfi['precio'].str.replace('€', '', regex=False)
#tambien creo las columnas precio anterior y descuento para los valores con precio del tipo  '€ 132.000€ 139.000(-5,0%)'
# Usar str.extract() para separar los valores en columnas individuales
dfi[['preciov', 'precio_anterior', 'descuento']] = dfi['precio'].str.extract(r'([\d\.,]+)\s+([\d\.,]+)\((-[\d,]+%)\)')
# Convertir las columnas 'precio' y 'precio_anterior' a tipo numérico
dfi['preciov'] = dfi['preciov'].str.replace('.', '').astype(float)
dfi['precio_anterior'] = dfi['precio_anterior'].str.replace('.', '').astype(float)
dfi.rename(columns={'precio': 'precio_1'}, inplace=True)

#elimino los edificios en venta
dfi= dfi[dfi['title'] != 'Edificio en Venta']
#quito m2 del valor de la superficie'

dfi.reset_index(drop=True)
#quito duplicados si los hubiera en cada dataframe
dfi.drop_duplicates()

# Supongamos que tu DataFrame se llama dfi
# Concatenar las columnas 'carc_todas' y 'atributos 2'
dfi['atr'] = dfi.apply(lambda row: f"{row['carc_todas']} | {row['atributos2']}", axis=1)
# Crear una nueva columna sin duplicados
dfi['atrib'] = dfi.apply(eliminar_duplicados, axis=1)
#Elimino las columnas atr,carac_todas y atributos2
dfi = dfi.drop(columns=['atr', 'carc_todas', 'atributos2'])

# Crear una nueva columna 'Lujo' con valor 1 si 'Lujo' o 'lujo' aparece en 'atributos', de lo contrario 0
dfi['lujo'] = dfi['atributos'].apply(lambda x: 1 if isinstance(x, str) and 'lujo' in x.lower() else 0)

In [222]:
# Aplicar la función a la columna 'atributos' y expandir el resultado en nuevas columnas
dfic = dfi.join(dfi['atrib'].apply(procesar_atributos).apply(pd.Series))
#elimino la columna nan que se genera cuando carc_todas tiene un nan
dfic = dfic.drop(columns=['nan'])

In [223]:
dfic['m2_constr']=dfic['Superficie'].str.replace('m²','')
dfic['m2_constr']=dfic['m2_constr'].str.replace('.','')

In [224]:


dfic['€/m2']=dfic['Precio del m²'].str.replace('€/m²','')
#pongo los diferentes atrib en columnas
dfic=dfic.reset_index(drop=True)
#creo una columna con la superficie comercial
# Llamar a la función pasando el DataFrame como argumento adicional 
dfic['sup_comercial'] = dfic.apply(lambda row: obtener_sup_comercial(row, dfic), axis=1)
dfic['tipo_inmueble'] = dfic.apply(lambda row: obtener_tipo_inmueble(row, dfic), axis=1)
# Eliminar columnas que contienen 'comercial' pero mantener 'sup_comercial'
dfic = dfic.drop(columns=[col for col in dfic.columns if 'comercial' in col and col != 'sup_comercial'])
dfic= dfic.drop(columns=dfic.filter(like='Tipo de inmueble').columns)
#la columna Contrato tiene siempre el valor Venta
dfic = dfic.drop(columns=['Precio del m²','Contrato'])


In [ ]:
# Aplicar la función a la columna 'title' y expandir el resultado en nuevas columnas
dfic[['title2', 'ciudad', 'distrito', 'cod_distrito', 'barrio', 'cod_barrio', 'zona']] = dfic['title'].apply(procesar_direccion)
# Aplicar la función solo a las filas donde el distrito es una cadena vacía
dfic.loc[dfic['distrito'] == 'Desconocido', ['distrito', 'barrio']] = dfic.loc[dfic['distrito'] == ''].apply(asignar_zona_distrito_barrio, axis=1)[['distrito', 'barrio']]
# Aplicar la función a la columna 'title2' y crear la nueva columna 'calle'
dfic['calle'] = dfic['title2'].apply(extraer_calle)
dfic=dfic.reset_index(drop=True)
# Renombrar la columna 'columna_vieja' a 'columna_nueva'
dfic.rename(columns={'Cocina': 'cocina_1'}, inplace=True)
mbl=MultiLabelBinarizer()
#resetear el indice, hacerlo cada poco
dfic=dfic.reset_index(drop=True)
#La columna badges tiene distintos atributos de las viviendas voy a hacer encoding para crear nuevas columnas, para poder aplicarlo primero la transformo en lsita
# Convertir cadenas de texto a listas reales
dfic['badges2'] = dfic['badges'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df_enc=pd.DataFrame(mbl.fit_transform(dfic.badges2),columns=mbl.classes_)
#concateno las nuevas columnas
df_enc.reset_index(drop=True, inplace=True)
df_final = pd.concat([dfic.reset_index(drop=True), df_enc], axis=1)
df_final.reset_index(drop=True)

In [197]:
#voy a unificar la columna precio y la columna lujo
df_final['Precio'] = df_final['Precio'].str.replace('€', '', regex=False)
# Renombrar la columna 'columna_vieja' a 'columna_nueva'
df_final.rename(columns={'Inmueble de lujo': 'lujo1'}, inplace=True)
df_final.rename(columns={'read_all': 'descripcion_detallada'}, inplace=True)
df_final['lujosa'] = np.where((df_final['lujo'] == 1) | (df_final['lujo1'].notna()), 1, 0)

#Le doy el valor Jardín privado y comunitario a la columna Jardín y elimino Jardín privado y comunitario
df_final.loc[df_final['Jardín privado y comunitario'] == 1, 'Jardín'] = 'Jardín privado y comunitario'

In [198]:
# Función para dividir la columna
def dividir_columna(valor):
    if 'kWh/m²' in valor:
        partes = valor.split('kWh/m²')
        consumo = partes[0].strip()
        letra = partes[1].strip()
    else:
        consumo = "desconocido"
        letra = valor   
    return consumo, letra

In [199]:
# Convertir todos los valores de la columna a cadenas de texto 
df_final['consumoenergia'] = df_final['consumoenergia'].astype(str)
# Aplicar la función y crear nuevas columnas 
df_final[['consumoCE_ano', 'letraCE']] = df_final['consumoenergia'].apply(lambda x: pd.Series(dividir_columna(x)))
df_final['letraCE'] = df_final['letraCE'].str.replace('año', '')
#puedo eliminar precio_1 y preciov y Cocina que no aporta
df_final = df_final.drop(columns=['precio_1', 'preciov','Cocina','lujo','lujo1','consumoenergia'])


In [200]:
df_Indomio = df_final[['title', 'url', 'Precio','precio_anterior', 'descuento', '€/m2', 'm2_constr', 'distrito', 'cod_distrito', 'barrio', 'cod_barrio', 'zona', 'calle', 
'title2', 'consumoCE_ano','letraCE', 'emisiones_co2', 'Tipología', 'Propiedad completa',  'Estado', 'Amueblado', 'Planta',
'Habitaciones', 'Dormitorios', 'Baños', 'Balcón','Terraza', 'Garaje, plazas de aparcamiento', 'Trastero', 'Ascensor',
'Servicio de portería', 'Acceso para minusválidos', 'Calefacción','Año de construcción',
'Plantas del edificio', 'Aire acondicionado', 'Jardín','Gastos de comunidad','Inmueble con ingresos',
'Alquiler con opción a compra', 'Disponibilidad',  'Usufructo', 'Nuda propiedad',
'Derecho de superficie', 'sup_comercial', 'tipo_inmueble','Armario empotrado', 'Buhardilla',
'Cancha de tenis', 'Carpintería exterior de doble vidrio/PVC',
'Carpintería exterior de doble vidrio/madera',
'Carpintería exterior de doble vidrio/metal',
'Carpintería exterior de triple vidrio/madera',
'Carpintería exterior de triple vidrio/metal',
'Carpintería exterior de vidrio/PVC',
'Carpintería exterior de vidrio/madera',
'Carpintería exterior de vidrio/metal', 'Chimenea', 'cocina_1', 'Fibra óptica', 'Hidromasaje',
'Instalación tv centralizada', 'Instalación tv individual', 'Exterior','Interior',
'Interior y exterior', 
'Orientación este', 'Orientación norte', 'Orientación norte, sur','Orientación oeste', 'Orientación sur', 'Orientación sur, este',
'Piscina', 'Portón eléctrico', 'Puerta blindada', 'Sistema de alarma','Videoportero',  'lujosa','descripcion_detallada']]


In [174]:
# Usar .loc para modificar el DataFrame original
df_Indomio.loc[:, 'm2_constr'] = pd.to_numeric(df_Indomio['m2_constr'], errors='coerce')


In [175]:

df_Indomio.query("m2_constr<20")

,title,url,Precio,precio_anterior,descuento,€/m2,Superficie,m2_constr,ciudad,distrito,cod_distrito,barrio,cod_barrio,zona,calle,title2,consumoCE_ano,letraCE,emisiones_co2,Tipología,Propiedad completa,Estado,Amueblado,Planta,Habitaciones,Dormitorios,Baños,Balcón,Terraza,"Garaje, plazas de aparcamiento",Trastero,Ascensor,Servicio de portería,Acceso para minusválidos,Calefacción,Año de construcción,Plantas del edificio,Aire acondicionado,Jardín,Gastos de comunidad,Inmueble con ingresos,Alquiler con opción a compra,Disponibilidad,Usufructo,Nuda propiedad,Derecho de superficie,sup_comercial,tipo_inmueble,Armario empotrado,Buhardilla,Cancha de tenis,Carpintería exterior de doble vidrio/PVC,Carpintería exterior de doble vidrio/madera,Carpintería exterior de doble vidrio/metal,Carpintería exterior de triple vidrio/madera,Carpintería exterior de triple vidrio/metal,Carpintería exterior de vidrio/PVC,Carpintería exterior de vidrio/madera,Carpintería exterior de vidrio/metal,Chimenea,cocina_1,Fibra óptica,Hidromasaje,Instalación tv centralizada,Instalación tv individual,Exterior,Interior,Interior y exterior,Orientación este,Orientación norte,"Orientación norte, sur",Orientación oeste,Orientación sur,"Orientación sur, este",Piscina,Portón eléctrico,Puerta blindada,Sistema de alarma,Videoportero,lujosa,descripcion_detallada


In [ ]:
df_Indomio.rename(columns={ 
'Servicio de portería': 'porteria', 
'Acceso para minusválidos': 'acceso_minusvalido',
'Garaje, plazas de aparcamiento':'Garaje' 
,'Año de construcción':'ano_construccion'
,'Plantas del edificio':'plantas_edificio'}, inplace=True)

In [53]:
determinantes=['el_', 'la_', 'los_', 'las_', 'un_', 'una_', 'unos_', 'unas_', 'de_','con_']
df_Indomio.columns = [
    col.strip().lower().replace(" ", "_")  # Normaliza espacios y minúsculas
    for col in df_Indomio.columns
]

df_Indomio.columns = [
    unidecode(reduce(lambda c, det: c.replace(det, ''), determinantes, col))  # Quita determinantes y acentos
    for col in df_Indomio.columns
]

In [ ]:
# Convertir la columna 'cod_distrito' a string 
df_final['cod_distrito'] = df_final['cod_distrito'].astype(str)
df_final['cod_barrio'] = df_final['cod_barrio'].astype(str)
# Convertir la columna 'cod_distrito' a string 
df_Indomio['cod_distrito'] = df_Indomio['cod_distrito'].astype(str)
df_Indomio['cod_barrio'] = df_Indomio['cod_barrio'].astype(str)

In [55]:
df_final.to_feather("df_final_ConTodasColumnas.ftr")

In [56]:
df_Indomio.to_feather("df_Indomio22122024.ftr")